<a href="https://colab.research.google.com/github/PRASANNA-416/Human-Activity-Detection-/blob/main/SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import pandas as pd

# Load the training CSV file
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Training_set.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing CSV file
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/testing_set.csv'
test_data = pd.read_csv(test_csv_path)


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CSV files
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Training_set.csv'
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/testing_set.csv'

train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

# Define the target image size and batch size
target_size = (224, 224)  # Adjust according to your requirements
batch_size = 32

# Create ImageDataGenerators with preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
    rotation_range=20,  # Randomly rotate images by 20 degrees
    width_shift_range=0.2,  # Randomly shift the width by 20%
    height_shift_range=0.2,  # Randomly shift the height by 20%
    horizontal_flip=True  # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/train',  # Directory containing the train images
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/test',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Check the class indices assigned by the generator
class_indices = test_generator.class_indices
print(class_indices)



Found 12100 validated image filenames belonging to 15 classes.
Found 500 validated image filenames belonging to 15 classes.
{'calling': 0, 'clapping': 1, 'cycling': 2, 'dancing': 3, 'drinking': 4, 'eating': 5, 'fighting': 6, 'hugging': 7, 'laughing': 8, 'listening_to_music': 9, 'running': 10, 'sitting': 11, 'sleeping': 12, 'texting': 13, 'using_laptop': 14}


In [4]:
import tensorflow as tf
import tensorflow_hub as hub

# Define the model
m = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/edgetpu/vision/mobilenet-edgetpu-v2-feature-vector/tiny/2',
        trainable=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(15, activation='softmax')
])


# Build the model with the default input shape
m.build((None, 224, 224, 3))  # Batch input shape.

# Print the model summary
print(m.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1, 1, 1280)        2517920   
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 15)                19215     
                                                                 
Total params: 2,537,135
Trainable params: 2,499,663
Non-trainable params: 37,472
_________________________________________________________________
None


In [5]:
print(m.get_config())

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 224, 224, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'keras_layer_input'}}, {'class_name': 'KerasLayer', 'config': {'name': 'keras_layer', 'trainable': True, 'dtype': 'float32', 'handle': 'https://tfhub.dev/google/edgetpu/vision/mobilenet-edgetpu-v2-feature-vector/tiny/2'}}, {'class_name': 'Flatten', 'config': {'name': 'flatten', 'trainable': True, 'dtype': 'float32', 'data_format': 'channels_last'}}, {'class_name': 'Dense', 'config': {'name': 'dense', 'trainable': True, 'dtype': 'float32', 'units': 15, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}]}


In [6]:
print(m.layers)

[<tensorflow_hub.keras_layer.KerasLayer object at 0x7f10ca6e3b80>, <keras.layers.reshaping.flatten.Flatten object at 0x7f105f482f80>, <keras.layers.core.dense.Dense object at 0x7f105f372320>]


In [9]:
layer = m.layers[0]  # Get the first layer (KerasLayer)

print(layer.get_config())  # Print the layer's configuration
# print(layer.trainable_weights)  # Print the trainable weights of the layer
# print(layer.non_trainable_weights)  # Print the non-trainable weights of the layer
print(layer.trainable)  # Print whether the layer is trainable or not


{'name': 'keras_layer', 'trainable': True, 'dtype': 'float32', 'handle': 'https://tfhub.dev/google/edgetpu/vision/mobilenet-edgetpu-v2-feature-vector/tiny/2'}
True


In [10]:
m.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
m.fit(train_generator, epochs=10,batch_size=batch_size)

Epoch 1/10
379/379 [==============================] - 3530s 9s/step - loss: 1.4489 - accuracy: 0.5422
Epoch 2/10
379/379 [==============================] - 2623s 7s/step - loss: 1.0703 - accuracy: 0.6565
Epoch 3/10
379/379 [==============================] - 2594s 7s/step - loss: 0.9505 - accuracy: 0.7033
Epoch 4/10
379/379 [==============================] - 2605s 7s/step - loss: 0.8564 - accuracy: 0.7288
Epoch 5/10
379/379 [==============================] - 2578s 7s/step - loss: 0.7933 - accuracy: 0.7443
Epoch 6/10
379/379 [==============================] - 2631s 7s/step - loss: 0.7224 - accuracy: 0.7650
Epoch 7/10
379/379 [==============================] - 2594s 7s/step - loss: 0.6931 - accuracy: 0.7755
Epoch 8/10
379/379 [==============================] - 2583s 7s/step - loss: 0.6524 - accuracy: 0.7883
Epoch 9/10
379/379 [==============================] - 2558s 7s/step - loss: 0.6109 - accuracy: 0.8013
Epoch 10/10
379/379 [==============================] - 2565s 7s/step - loss: 0.557

In [11]:


# Evaluate the model on the test generator
test_loss, test_accuracy = m.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

16/16 [==============================] - 119s 8s/step - loss: 1.3335 - accuracy: 0.6440
Test Loss: 1.3334535360336304
Test Accuracy: 0.6439999938011169


In [12]:
m.save_weights('squeezeNet.h5')

In [13]:
from google.colab import files
files.download('squeezeNet.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>